<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Dataframes II
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Планы выполнения задач
+ Оптимизация соединений и группировок
+ Управление схемой данных
+ Оптимизатор запросов Catalyst

## Планы выполнения задач

Любой `job` в Spark SQL имеет под собой план выполнения, кототорый генерируется на основе написанно запроса. План запроса содержит операторы, которые затем превращаются в Java код. Поскольку одну и ту же задачу в Spark SQL можно выполнить по-разному, полезно смотреть в планы выполнения, чтобы, например:
+ убрать лишние shuffle
+ убедиться, чтот тот или иной оператор будет выполнен на уровне источника, а не внутри Spark
+ понять, как будет выполнен `join`

Планы выполнения доступны в двух видах:
+ метод `explain()` у DF
+ на вкладке SQL в Spark UI

Прочитаем датасет [Airport Codes](https://datahub.io/core/airport-codes):

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Используем метод `explain`, чтобы посмотреть план запроса. Наиболее интересным является физический план, т.к. он отражает фактически алгоритм обработки данных. В данном случае в плане присутствует единственный оператор `FileScan csv`:

In [ ]:
airports.explain(extended = true)

Если остальные планы не нужны, можно показать только физический:

In [ ]:
import org.apache.spark.sql.Dataset

airports.queryExecution.executedPlan.treeString

def printPhysicalPlan[_](ds: Dataset[_]): Unit = {
    println(ds.queryExecution.executedPlan.treeString)
}

Также есть возмжность получить эту информацию в виде JSON:

In [ ]:
airports.queryExecution.executedPlan.toJSON

Выполним `filter` и проверим план выполнения. Читать план нужно снизу вверх. В плане появился новый оператор `filter`

In [ ]:
printPhysicalPlan(airports.filter('type === "small_airport"))

Выполним агрегацию и проверим план выполнения. В нем появляется три оператора: 2 `HashAggregate` и `Exchange hashpartitioning`.

Первый `HashAggregate` содержит функцию `partial_count(1)`. Это означает, что внутри каждого воркера произойдет подсчет строк по каждому ключу. Затем происходит `shuffle` по ключу агрегата, после которого выполняется еще один `HashAggregate` с функцией `count(1)`. Использование двух `HashAggregate` позволяет сократить количество передаваемых данных по сети.

In [ ]:
printPhysicalPlan(airports.filter('type === "small_airport").groupBy('iso_country).count)

При необходимости мы можем почитать ~~перед сном~~ сгенерированный ~~теплый ламповый~~ java код:

In [ ]:
import org.apache.spark.sql.execution.command.ExplainCommand

val grouped = airports.filter('type === "small_airport").groupBy('iso_country).count


def printCodeGen[_](ds: Dataset[_]): Unit = {
    val logicalPlan = ds.queryExecution.logical
    val codeGen = ExplainCommand(logicalPlan, extended = true, codegen = true)
    spark.sessionState.executePlan(codeGen).executedPlan.executeCollect().foreach {
      r => println(r.getString(0))
    }
}

printCodeGen(grouped)

<img align="right" width="200" height="200" src="https://cs5.pikabu.ru/post_img/big/2015/12/11/7/1449830295198229367.jpg">

### Выводы:
+ Spark составляет физический план выполнения запроса на основании написанного вами кода
+ Изучив план запроса, можно понять, какие операторы будут применены в ходе обработки ваших данных
+ План выполнения запроса - один из основных инструментов оптимизации запроса

## Оптимизация соединений и группировок
При выполнении `join` двух DF важно следовать рекомендациям:
+ фильтровать данные до join'а
+ использовать equ join 
+ если можно путем увеличения количества данных применить equ join вместо non-equ join'а, то делать именно так
+ всеми силами избегать cross-join'ов
+ если правый DF помещается в памяти worker'а, использовать broadcast()

### Виды соединений
+ **BroadcastHashJoin**
  - equ join
  - broadcast
+ **SortMergeJoin**
  - equ join
  - sortable keys
+ **BroadcastNestedLoopJoin**
  - non-equ join
  - using broadcast
+ **CartesianProduct**
  - non-equ join
  
[Optimizing Apache Spark SQL Joins: Spark Summit East talk by Vida Ha](https://youtu.be/fp53QhSfQcI)

Подготовим два датасета:

In [ ]:
val left = airports.select('type, 'ident, 'iso_country)
val right = airports.groupBy('type).count

### BroadcastHashJoin
+ работает, когда условие - равенство одного или нескольких ключей
+ работает, когда один из датасетов небольшой и полностью вмещается в память воркера
+ оставляет левый датасет как есть
+ копирует правый датасет на каждый воркер
+ составляет hash map из правого датасета, где ключ - кортеж из колонок в условии соединения
+ итерируется по левому датасета внутри каждой партиции и проверяет наличие ключей в HashMap
+ может быть автоматически использован, либо явно через `broadcast(df)`

In [ ]:
import org.apache.spark.sql.functions.broadcast
val result = left.join(broadcast(right), Seq("type"), "inner")

printPhysicalPlan(result)

### SortMergeJoin
+ работает, когда ключи соединения в обоих датасета являются сортируемыми
+ репартиционирует оба датасета в 200 партиций по ключу (ключам) соединения
+ сортирует партиции каждого из датасетов по ключу (ключам) соединения
+ Используя сравнение левого и правого ключей, обходит каждую пару партиций и соединяет строки с одинаковыми ключами

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

val result = left.join(right, Seq("type"), "inner")

printPhysicalPlan(result)

### BroadcastNestedLoopJoin
+ работает, когда один из датасетов небольшой и полностью вмещается в память воркера
+ оставляет левый датасет как есть
+ копирует правый датасет на каждый воркер
+ проходится вложенным циклом по каждой партиции левого датасета и копией правого датасета и проверяет условие
+ может быть автоматически использован, либо явно через `broadcast(df)`

In [ ]:
import org.apache.spark.sql.functions.{ expr, udf, col }

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

// Не смотря на то, что UDF сравнивает два ключа, Spark ничего про нее не знает
// и не может применить BroadcastHashJoin или SortMergeJoin
val compare_udf = udf { (leftVal: String, rightVal: String) => leftVal == rightVal }

val joinExpr = compare_udf(col("left.type"), col("right.type"))

val result = left.as("left").join(broadcast(right).as("right"), joinExpr, "inner")

printPhysicalPlan(result)

### CartesianProduct
+ Создает пары из каждой партиции левого датасета с каждой партицией правого датасета, релоцирует каждую пару на один воркер и проверяет условие соединения
+ на выходе создает N*M партиций
+ работает медленнее остальных и часто приводит к ООМ воркеров

In [ ]:
import org.apache.spark.sql.functions.{ expr, udf, col }

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

// Не смотря на то, что UDF сравнивает два ключа, Spark ничего про нее не знает
// и не может применить BroadcastHashJoin или SortMergeJoin
val compare_udf = udf { (leftVal: String, rightVal: String) => leftVal == rightVal }

val joinExpr = compare_udf(col("left.type"), col("right.type"))

val result = left.as("left").join(right.as("right"), joinExpr, "inner")

printPhysicalPlan(result)
println(
    s"""Partition summary: 
    left=${left.rdd.getNumPartitions}, 
    right=${right.rdd.getNumPartitions}, 
    result=${result.rdd.getNumPartitions}""")

### Снижение количества shuffle
В ряде случаев можно уйти от лишних `shuffle` операций при выполнении соединения. Для этого оба DF должны иметь одинаковое партиционирование - одинаковое количество партиций и ключ партиционирования, совпадающий с ключом соединения.

Разница между планами выполнения будет хорошо видна в Spark UI на графе выполнения в Jobs и плане выполнения в SQL

In [ ]:
spark.time { 
    val left = airports
    val right = airports.groupBy('type).count

    val joined = left.join(right, Seq("type"))

    joined.count
}

In [ ]:
spark.time { 
    val airportsRep = airports.repartition(200, col("type"))
    val left = airportsRep
    val right = airportsRep.groupBy('type).count

    val joined = left.join(right, Seq("type"))

    joined.count
}

### Выводы:
+ В Spark используются 4 вида соединений: `BroadcastHashJoin`, `SortMergeJoin`, `BroadcastNestedLoopJoin`, `CartesianProduct`
+ Выбор алгоритма основывается на условии соединения и размере датасетов
+ `CartesianProduct` обладает самой низкой вычислительной эффективностью и его по возможности стоит избегать

## Управление схемой данных
В DF API каждая колонка имеет свой тип. Он может быть:
+ скаляром - `StringType`, `IntegerType` и т. д.
+ массивом - `ArrayType(T)`
+ словарем `MapType(K, V)`
+ структурой - `StructType()`

DF целиком также имеет схему, описанную с помощью класса `StructType`

Посмотреть список колонок можно с помощью атрибута `columns`:

In [ ]:
airports.columns

Схема DF доступна через атрибут `schema`

In [ ]:
import org.apache.spark.sql.types._
val schema: StructType = airports.schema

`apply()` метод возвращает поле структуры по имени, как в словаре

In [ ]:
val field: StructField = schema("ident")

`StructField` обладает атрибутами `name` и `dataType`:

In [ ]:
val name: String = field.name

val fieldType: DataType = field.dataType

fieldType match {
    case f: StringType => println("This is string")
    case _ => println("This is not string!")
}

Метод `simpleString` можно использовать, чтобы получить DDL схемы в виде строки:

In [ ]:
fieldType.simpleString

In [ ]:
val airportSchema = schema.simpleString

Схема может быть создана из `case class`:

In [ ]:
import org.apache.spark.sql.types._

case class Airport(
    ident: String,
    `type`: String,
    name: String,
    elevation_ft: Int,
    continent: String,
    iso_country: String,
    iso_region: String,
    municipality: String,
    gps_code: String,
    iata_code: String,
    local_code: String,
    coordinates: String
)

import org.apache.spark.sql.catalyst.ScalaReflection
val schemaFromClass = ScalaReflection.schemaFor[Airport].dataType.asInstanceOf[StructType]

Схема может быть использована:
+ при чтении источника
+ при работе с JSON

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "false")
val airports = spark.read.options(csvOptions).schema(schemaFromClass).csv("/tmp/datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

In [ ]:
import org.apache.spark.sql.functions._

val parseJson = from_json(col("value"), schemaFromClass).alias("s")

val jsoned = airports.toJSON

val withColumns = jsoned.select(parseJson).select(col("s.*"))

withColumns.show(1, 200, true)
withColumns.printSchema

Схема может быть создана вручную:

In [ ]:
val someSchema = 
    StructType(
        List(
            StructField("foo", StringType),
            StructField("bar", StringType),
            StructField(
                        "boo", 
                        StructType(
                            List(
                                StructField("x", IntegerType),
                                StructField("y", BooleanType)
                                )
                            )
                       )
        
        )
    )

someSchema.printTreeString()

Схема также может быть получена из JSON строки:

In [ ]:
val jsoned = airports.toJSON

val firstLine = jsoned.head

spark.range(1).select(schema_of_json(lit(firstLine))).head

Чтобы изменить тип колонки, следует использовать метод `cast`. Данная операция может как возвращать `null`, так и бросать исключение

In [ ]:
airports.select('elevation_ft.cast("string")).printSchema
airports.select('elevation_ft.cast("string")).show(1, false)

In [ ]:
airports.select('type.cast("float")).printSchema
airports.select('type.cast("float")).show(1, false)

### Выводы:
+ Spark использует схемы для описания типов колонок, схемы всего DF, чтения источников и для работы с JSON
+ Схема представляет собой инстанс класса `StructType`
+ Колонки в Spark могут иметь любой тип. При этом вложенность словарей, массивов и структур не ограничена

## Оптимизатор запросов Catalyst
Catalyst выполняет оптимизацию запросов с целью ускорения их выполнения и применяет следующие методы:
 + Column projection
 + Partition pruning
 + Predicate pushdown
 + Constant folding
 
 Подготовим датасет для демонстрации работы Catalyst:

In [ ]:
airports
    .write
    .format("parquet")
    .partitionBy("iso_country")
    .mode("overwrite")
    .save("/tmp/datasets/airports.parquet")

val airportPq = spark.read.parquet("/tmp/datasets/airports.parquet")

### Column projection
Данный механизм позволяет избегать вычитывания ненужных колонок при работе с источниками

In [ ]:
spark.time { 
    val selected = airportPq.select('ident)
    selected.cache
    selected.count
    selected.unpersist
    printPhysicalPlan(selected)
}

In [ ]:
spark.time { 
    val selected = airportPq
    selected.cache
    selected.count
    selected.unpersist
    printPhysicalPlan(selected)
}

### Partition pruning
Данный механизм позволяет избежать чтения ненужных партиций

In [ ]:
spark.time { 
    val filtered = airportPq.filter('iso_country === "RU")
    filtered.count
    printPhysicalPlan(filtered)
}

### Predicate pushdown
Данный механизм позволяет "протолкнуть" условия фильтрации данных на уровень datasource

In [ ]:
spark.time { 
    val filtered = airportPq.filter('iso_region === "RU")
    filtered.count
    printPhysicalPlan(filtered)
}

### Simplify casts
Данный механизм убирает ненужные `cast`

In [ ]:
val result = spark.range(0,10).select('id.cast("long"))
printPhysicalPlan(result)

In [ ]:
val result = spark.range(0,10).select('id.cast("int"))
printPhysicalPlan(result)

### Constant folding
Данный механизм сокращает количество констант, используемых в физическом плане

In [ ]:
val result = spark.range(0,10).select((lit(3) >  lit(0)).alias("foo"))
printPhysicalPlan(result)

In [ ]:
val result = spark.range(0,10).select(('id >  0).alias("foo"))
printPhysicalPlan(result)

### Combine filters
Данный механизм объединяет фильтры

In [ ]:
val result = spark.range(0,10).filter('id > 0).filter('id !== 5).filter('id < 10)
printPhysicalPlan(result)